In [1]:
import os
import PIL.Image
from tensorflow.python.keras.utils.generic_utils import Progbar
from tf_agents.environments import suite_gym, parallel_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.trajectories.trajectory import Trajectory
from tf_agents.networks import actor_distribution_network
from tf_agents.drivers import dynamic_episode_driver, dynamic_step_driver
from tf_agents.policies import random_tf_policy
import tensorflow as tf
import numpy as np
from reinforcement_learning.sac_training import NumberOfSafetyViolations
from reinforcement_learning import sac_training
from reinforcement_learning import labeling_functions

In [2]:
py_env = suite_gym.load('BipedalWalker-v2')
py_env.render(mode='human')
py_env.reset()


TimeStep(step_type=array(0, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([ 2.7449459e-03,  1.6034379e-05, -2.0907968e-03, -1.6000107e-02,
        9.2666708e-02,  4.8536574e-03,  8.5959542e-01, -2.4567314e-03,
        1.0000000e+00,  3.2936331e-02,  4.8534656e-03,  8.5344189e-01,
       -3.3024745e-03,  1.0000000e+00,  4.4081330e-01,  4.4581941e-01,
        4.6142203e-01,  4.8954940e-01,  5.3410190e-01,  6.0246003e-01,
        7.0914775e-01,  8.8593036e-01,  1.0000000e+00,  1.0000000e+00],
      dtype=float32))

In [3]:
bad_state_detection = lambda trajectory: print("bad state!: {}".format(trajectory.observation[..., 0]))\
    if tf.math.abs(trajectory.observation[...,0]) > np.pi / 3 else None

tf_env = tf_py_environment.TFPyEnvironment(py_env)
policy = random_tf_policy.RandomTFPolicy(time_step_spec=tf_env.time_step_spec(), action_spec=tf_env.action_spec())
dynamic_episode_driver.DynamicEpisodeDriver(tf_env, policy, num_episodes=15,
                                            observers=[bad_state_detection, lambda _: py_env.render(mode='human')]).run()

bad state!: [1.0735494]
bad state!: [1.1378732]
bad state!: [1.174754]
bad state!: [1.2165273]
bad state!: [1.2634566]
bad state!: [1.31943]
bad state!: [1.3697844]
bad state!: [1.4257612]
bad state!: [1.4486148]
bad state!: [1.4686921]
bad state!: [1.4952875]
bad state!: [1.5231817]
bad state!: [1.5553536]
bad state!: [1.6080016]
bad state!: [1.6468967]
bad state!: [1.0722704]
bad state!: [1.0935936]
bad state!: [1.1474162]
bad state!: [1.2080934]
bad state!: [1.2711015]
bad state!: [1.281411]
bad state!: [1.2803789]
bad state!: [1.2787379]
bad state!: [1.2594597]
bad state!: [1.2564876]
bad state!: [1.2487124]
bad state!: [1.2456635]
bad state!: [1.2509632]
bad state!: [1.25248]
bad state!: [1.2463536]
bad state!: [1.2301029]
bad state!: [1.2135035]
bad state!: [1.2022425]
bad state!: [1.1893189]
bad state!: [1.1752688]
bad state!: [1.1632671]
bad state!: [1.1410406]
bad state!: [1.1098207]
bad state!: [1.0751128]
bad state!: [1.0602597]
bad state!: [1.055596]
bad state!: [1.0899857]

(TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 24), dtype=float32, numpy=
 array([[ 2.7459345e-03,  9.9786403e-06, -1.3011862e-03, -1.6000066e-02,
          9.2471495e-02,  3.0206335e-03,  8.5980916e-01, -9.9893811e-04,
          1.0000000e+00,  3.2791901e-02,  3.0205012e-03,  8.5354382e-01,
         -2.0552513e-03,  1.0000000e+00,  4.4081339e-01,  4.4581950e-01,
          4.6142212e-01,  4.8954949e-01,  5.3410202e-01,  6.0246021e-01,
          7.0914787e-01,  8.8593054e-01,  1.0000000e+00,  1.0000000e+00]],
       dtype=float32)>),
 ())

In [212]:
def bad_state_detection(trajectory):
    # if trajectory.reward[..., 0] <= -100:
    if trajectory.observation[..., 0] < -1. or trajectory.observation[..., 0] > 1:
        py_env.render(mode='human')
        print(trajectory.observation)
        return True
    else:
        return False

walk = True
while walk:
    action = policy.action(time_step=tf_env.current_time_step())
    time_step = tf_env.step(action)
    walk = not bad_state_detection(time_step)


tf.Tensor(
[[-1.139621   -0.15121226  0.19145721 -0.11511189 -0.5889525   0.6279823
  -0.6495559  -1.          0.          0.1954412  -0.14670914  0.18718338
   1.0000001   0.          0.23256062  0.23520169  0.24343319  0.25827238
   0.28177702  0.31784084  0.37412626  0.4673918   0.6417984   1.        ]], shape=(1, 24), dtype=float32)


In [3]:
from reinforcement_learning import labeling_functions

labeling_function = labeling_functions['BipedalWalker-v2']
safety_violations = NumberOfSafetyViolations(labeling_function)
progressbar = Progbar(target=None, interval=0.5, stateful_metrics=['violation'])

tf_env = tf_py_environment.TFPyEnvironment(py_env)
policy = random_tf_policy.RandomTFPolicy(
    time_step_spec=tf_env.time_step_spec(),
    action_spec=tf_env.action_spec())
dynamic_episode_driver.DynamicEpisodeDriver(
    tf_env,
    policy,
    num_episodes=15,
    observers=[safety_violations,
               lambda _: progressbar.add(
                   1, [('violation', safety_violations.average())]),
               lambda _: py_env.render(mode='human')]
).run()

safety_violations._num_episodes

  10233/Unknown - 175s 17ms/step - violation: 7.4667  

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=16.0>

In [2]:
# Parallel environments.
num_parallel_environments = 4
tf_env = tf_py_environment.TFPyEnvironment(
    parallel_py_environment.ParallelPyEnvironment(
    [lambda : suite_gym.load('BipedalWalker-v2')] * num_parallel_environments))
tf_env.reset()

TimeStep(step_type=<tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 0, 0, 0], dtype=int32)>, reward=<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>, discount=<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>, observation=<tf.Tensor: shape=(4, 24), dtype=float32, numpy=
array([[ 2.7475352e-03, -2.3226251e-06,  1.8068074e-04, -1.5999984e-02,
         9.2117779e-02, -2.3843770e-04,  8.6017948e-01,  1.5795087e-03,
         1.0000000e+00,  3.2523964e-02, -2.3842111e-04,  8.5372776e-01,
         1.6862502e-04,  1.0000000e+00,  4.4081393e-01,  4.4582003e-01,
         4.6142268e-01,  4.8955008e-01,  5.3410268e-01,  6.0246092e-01,
         7.0914876e-01,  8.8593167e-01,  1.0000000e+00,  1.0000000e+00],
       [ 2.7474831e-03, -6.6567359e-06,  5.1782356e-04, -1.5999954e-02,
         9.2057437e-02, -6.8335218e-04,  8.6021441e-01,  1.9081774e-03,
         1.0000000e+00,  3.2463934e-02, -6.8330392e-04,  8.5376263e-01,
     

In [3]:
labeling_function = labeling_functions['BipedalWalker-v2']

policy = random_tf_policy.RandomTFPolicy(
    time_step_spec=tf_env.time_step_spec(),
    action_spec=tf_env.action_spec())

safety_violations = NumberOfSafetyViolations(labeling_function)

dynamic_step_driver.DynamicStepDriver(
    tf_env,
    policy,
    num_steps=5000,
    observers=[safety_violations]
).run()

print('Safety violations')
print('episodes', safety_violations._num_episodes)
print('result=', safety_violations.result())
print('average=', safety_violations.average())

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
Safety violations
episodes <tf.Variable 'num_episodes:0' shape=() dtype=float32, numpy=1.0>
result= <tf.Variable 'num_violations:0' shape=() dtype=float32, numpy=149.0>
average= tf.Tensor(149.0, shape=(), dtype=float32)


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


In [1]:
import importlib
from tf_agents.environments import suite_gym
from reinforcement_learning import labeling_functions
from reinforcement_learning import sac_training

importlib.reload(sac_training)

learner = sac_training.SACLearner(
    env_name='BipedalWalker-v2',
    env_suite=suite_gym,
    num_iterations=int(1e6),
    num_parallel_environments=8,
    labeling_function=labeling_functions['BipedalWalker-v2']
)

Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.


Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.


In [ ]:
learner.train_and_eval()

Checkpoint loaded! global_step=2500
Initialize replay buffer...
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
Start training...
   2495/1000000 [..............................] - ETA: 5:09:22 - loss: 6817.6499 - replay_buffer_frames: 187576.0000   

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./saves/BipedalWalker-v2/stochastic_policy/assets
INFO:tensorflow:Assets written to: ./saves/BipedalWalker-v2/stochastic_policy/assets
INFO:tensorflow:Assets written to: ./saves/BipedalWalker-v2/stochastic_policy/assets
Exception in thread eval:
Traceback (most recent call last):
  File "/home/florentdelgrange/anaconda3/envs/vae-mdp/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/florentdelgrange/anaconda3/envs/vae-mdp/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
TypeError: eval() argument after * must be an iterable, not numpy.int64
INFO:tensor

In [5]:
learner.safety_violations.result()


<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>